# LEGO Bricks ML Vision - Model Generation Notebook

This notebook provides a one-click solution to reproduce all trained models from the project:
1. Bricks detection model
2. Studs detection model
3. Multiclass brick detection model

Each model will be trained using the default configurations and the outputs will be saved as zip files.

In [ ]:
# Install required packages
!pip install rich pathlib

In [ ]:
import os
import subprocess
from pathlib import Path
import sys
from rich.console import Console
from rich.logging import RichHandler
import logging
import shutil

# Configure rich logging
logging.basicConfig(
    level=logging.INFO,
    format="%(message)s",
    handlers=[RichHandler(rich_tracebacks=True)]
)

console = Console()

# Repository URL
REPO_URL = 'https://github.com/MiguelDiLalla/LEGO_Bricks_ML_Vision.git'

def run_command(cmd, cwd=None, description=None):
    """Run a command and capture output with proper logging"""
    if description:
        logging.info(f"🚀 {description}")
    
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        shell=True,
        cwd=cwd
    )
    
    # Stream output in real-time
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())
            
    # Get the return code
    return_code = process.poll()
    
    if return_code != 0:
        error = process.stderr.read()
        raise subprocess.CalledProcessError(return_code, cmd, error)
    
    return return_code

In [ ]:
# Clean previous workspace if exists
workspace_dir = Path("workspace")
if workspace_dir.exists():
    shutil.rmtree(workspace_dir)
workspace_dir.mkdir()

# Clone repository
run_command(
    f"git clone {REPO_URL} .",
    cwd=workspace_dir,
    description="Cloning repository"
)

# Install requirements
run_command(
    f"{sys.executable} -m pip install -r requirements.txt",
    cwd=workspace_dir,
    description="Installing requirements"
)

In [ ]:
# Train bricks detection model
logging.info("📦 Starting bricks model training")
try:
    run_command(
        f"{sys.executable} train_singleclass.py train --mode bricks --yes",
        cwd=workspace_dir,
        description="Training bricks detection model"
    )
    logging.info("✅ Bricks model training completed")
except Exception as e:
    logging.error(f"❌ Bricks model training failed: {str(e)}")
    raise

In [ ]:
# Train studs detection model
logging.info("📦 Starting studs model training")
try:
    run_command(
        f"{sys.executable} train_singleclass.py train --mode studs --yes",
        cwd=workspace_dir,
        description="Training studs detection model"
    )
    logging.info("✅ Studs model training completed")
except Exception as e:
    logging.error(f"❌ Studs model training failed: {str(e)}")
    raise

In [ ]:
# Train multiclass model
logging.info("📦 Starting multiclass model training")
try:
    run_command(
        f"{sys.executable} train_multiclass.py train --yes",
        cwd=workspace_dir,
        description="Training multiclass detection model"
    )
    logging.info("✅ Multiclass model training completed")
except Exception as e:
    logging.error(f"❌ Multiclass model training failed: {str(e)}")
    raise

In [ ]:
# Display results location
parent_dir = workspace_dir.parent
logging.info("\n🎉 Training pipeline completed!")
logging.info("\nModel archives can be found in:")
for zip_file in parent_dir.glob("*.zip"):
    logging.info(f"📁 {zip_file}")